## 1. Setup: Import Libraries and Define Dataset Size

Import necessary Python libraries and define the scale of our synthetic dataset:
- **pandas**: For DataFrame manipulation and CSV export
- **random**: For generating random demographic and rating data
- **Faker**: For creating realistic fake data (movie titles, reviews, dates)
- **Dataset size**: 500 users, 500 movies, 1000 ratings


In [1]:
import pandas as pd
import random
from faker import Faker

fake = Faker()
n_users = 500
n_movies = 500
n_ratings = 1000


## 2. Generate Users Table

Create synthetic user demographic data:
- **user_id**: Unique identifier (1-500)
- **gender**: Randomly assigned from ['Male', 'Female', 'Other']
- **location**: Random selection from 8 global cities
- **age**: Random integer between 18-70 years

This simulates a diverse user base across demographics and geographies.


In [2]:
users = []
genders = ['Male', 'Female', 'Other']
locations = ['London', 'New York', 'Sydney', 'Delhi', 'Paris', 'Berlin', 'Toronto', 'Tokyo']

for i in range(n_users):
    users.append([
        i+1,
        random.choice(genders),
        random.choice(locations),
        random.randint(18, 70)
    ])
users_df = pd.DataFrame(users, columns=['user_id', 'gender', 'location', 'age'])


## 3. Generate Movies Table

Create synthetic movie catalog data:
- **movie_id**: Unique identifier (1-500)
- **title**: Realistic 3-word movie titles generated using Faker
- **genre**: Random selection from 8 popular genres
- **release_year**: Random year between 1980-2025
- **duration_minutes**: Random duration between 80-180 minutes

This provides a realistic and varied movie dataset for analysis.


In [3]:
genres = ['Action', 'Comedy', 'Drama', 'Thriller', 'Romance', 'Sci-Fi', 'Animation', 'Horror']
movies = []
for i in range(n_movies):
    movies.append([
        i+1,
        fake.sentence(nb_words=3).replace('.', ''),
        random.choice(genres),
        random.randint(1980, 2025),
        random.randint(80, 180)
    ])
movies_df = pd.DataFrame(movies, columns=['movie_id', 'title', 'genre', 'release_year', 'duration_minutes'])


## 4. Generate Ratings Table (Composite Primary Key)

Create user-movie rating records with uniqueness enforcement:
- **Composite Key**: Each (user_id, movie_id) pair is unique—no duplicate ratings
- **used_pairs set**: Tracks existing combinations to prevent duplicates
- **rating**: Random score from 1-5 stars
- **review_text**: Realistic 8-word review using Faker
- **date_rated**: Random date within the last 2 years
- **minutes_watched**: Random viewing duration (30-180 minutes)

This ensures database integrity and mimics real-world rating platform behavior.


In [4]:
# Ratings (with composite key)
used_pairs = set()
ratings = []
tries = 0

while len(ratings) < n_ratings and tries < n_ratings * 2:
    user_id = random.randint(1, n_users)
    movie_id = random.randint(1, n_movies)
    pair = (user_id, movie_id)
    if pair in used_pairs:
        tries += 1  # Avoid infinite loop
        continue
    used_pairs.add(pair)
    rating = random.randint(1, 5)
    review_text = fake.sentence(nb_words=8)
    date_rated = fake.date_between(start_date='-2y', end_date='today').strftime('%Y-%m-%d')
    minutes_watched = random.randint(30, 180)
    ratings.append([
        user_id,
        movie_id,
        rating,
        review_text,
        date_rated,
        minutes_watched
    ])

ratings_df = pd.DataFrame(ratings, columns=[
    'user_id', 'movie_id', 'rating', 'review_text', 'date_rated', 'minutes_watched'
])


## 5. Simulate Missing Data

Introduce realistic data quality issues by randomly inserting NULL values:
- **Users**: 8 missing values across gender, location, or age
- **Movies**: 9 missing values across genre, release_year, or duration
- **Ratings**: 13 missing values across rating, review, date, or minutes_watched

This simulates real-world data imperfections and enables data cleaning practice.


In [5]:
# Users (missing values)
for _ in range(8):
    idx = random.randint(0, n_users-1)
    col = random.choice(['gender', 'location', 'age'])
    users_df.at[idx, col] = None  # or "" for blank

# Movies (missing values)
for _ in range(9):
    idx = random.randint(0, n_movies-1)
    col = random.choice(['genre', 'release_year', 'duration_minutes'])
    movies_df.at[idx, col] = None

# Ratings (missing values)
for _ in range(13):
    idx = random.randint(0, n_ratings-1)
    col = random.choice(['rating', 'review_text', 'date_rated', 'minutes_watched'])
    ratings_df.at[idx, col] = None


## 6. Export DataFrames to CSV

Save all three tables as CSV files for database import:
- `users.csv`
- `movies.csv`
- `ratings.csv`

**Note**: `index=False` prevents pandas from adding an extra index column to the CSV.

These files will be imported into SQLite using DB Browser for SQLite.


In [6]:
users_df.to_csv('users.csv', index=False)
movies_df.to_csv('movies.csv', index=False)
ratings_df.to_csv('ratings.csv', index=False)


## 7. Data Quality Check: Count Missing Values

Display the count of NULL values in each table to verify data preparation:
- Confirms missing data was successfully introduced
- Documents baseline data quality before preprocessing
- Helps plan data cleaning strategies (e.g., imputation with 0 or 'unknown')

**Output**: Shows missing value counts per column for each table.


In [7]:
print(users_df.isnull().sum())
print(movies_df.isnull().sum())
print(ratings_df.isnull().sum())

user_id     0
gender      3
location    3
age         2
dtype: int64
movie_id            0
title               0
genre               2
release_year        2
duration_minutes    5
dtype: int64
user_id            0
movie_id           0
rating             4
review_text        6
date_rated         2
minutes_watched    1
dtype: int64
